\
            # 35. 并发与网络基础（Concurrency & Networking Basics）

            用一节课把“并发模型怎么选”和“网络通信的基本套路”讲清楚：
- 并发/并行、I/O/CPU 密集如何判断
- TCP/UDP/HTTP 的直觉与常见坑
- 协议分帧（framing）、超时/重试/幂等/背压等工程必备

            > 约定：Python 3.8；示例尽量只用标准库；代码块可直接运行（第三方依赖会做可选降级）。


## 前置知识

- 字符串/字节串（str/bytes）
- 函数与异常基础
- 列表/字典、with 语句


## 知识点地图

- 1. 并发 vs 并行：先看瓶颈
- 2. 线程/进程/协程：成本与边界
- 3. TCP/UDP/socket：字节流 vs 数据报
- 4. 粘包/半包：为什么要 framing（分帧）
- 5. HTTP/JSON：请求-响应模型（最小可运行）
- 6. 超时、重试、幂等、背压：工程化必修


## 自检清单（学完打勾）

- [ ] 能区分并发/并行、I/O 密集/CPU 密集
- [ ] 知道线程/进程/协程的适用场景与边界
- [ ] 理解 TCP 是字节流：必须设计分帧
- [ ] 能为网络调用设置超时并合理重试
- [ ] 理解幂等、背压、限流在分布式系统里的意义


In [ ]:
\
from pathlib import Path

ART = Path('_nb_artifacts')
ART.mkdir(exist_ok=True)
print('artifacts dir:', ART.resolve())


## 知识点 1：并发 vs 并行：先看瓶颈

- 并发（Concurrency）：同一时间段内处理多个任务（交错执行）。
- 并行（Parallelism）：同一时刻真正同时执行（多核）。
- I/O 密集：大多数时间在等网络/磁盘；常用线程或异步。
- CPU 密集：大多数时间在算；常用多进程（绕开 GIL）。

实战决策顺序：
1) 找瓶颈（CPU 还是 I/O） 2) 明确 SLA（吞吐/延迟） 3) 明确失败语义（超时、重试、幂等）


## 知识点 2：线程/进程/协程：成本与边界

- 线程：共享内存，切换成本低；要用锁保护共享状态；适合 I/O 并发。
- 进程：内存隔离，稳定性更好；通信要序列化；适合 CPU 并行。
- 协程：单线程内可控切换；对 I/O 友好；配合事件循环（如 asyncio）。

关键：模型不是银弹；正确性（同步/超时/资源回收）永远优先。


## 知识点 3：TCP/UDP/socket：字节流 vs 数据报

- TCP：面向连接、可靠、有序；应用层看到“字节流”，不保留消息边界。
- UDP：无连接、尽力而为；保留消息边界（一个 datagram）。
- socket 工程要点：超时、最大消息、连接上限、资源回收。


## 知识点 4：粘包/半包：为什么要 framing（分帧）

TCP 只保证“按顺序送达字节”，不保证你 send 的一条 == 对方 recv 的一条。

常见分帧方案：
- 分隔符（文本协议）：例如以 
 结尾；要限制最大行长度。
- 长度前缀（二进制/JSON 常用）：先发 4 字节长度，再发正文。
- 定长消息：简单但不灵活。


## 知识点 5：HTTP/JSON：请求-响应模型（最小可运行）

HTTP 是最常见的应用层协议：方法、路径、头、体；状态码 2xx/4xx/5xx。

下面示例在本机起一个 HTTP 服务并用 urllib 请求，避免依赖外网。


In [ ]:
import json
from http.server import BaseHTTPRequestHandler, HTTPServer
from threading import Thread
from urllib.request import urlopen


class Handler(BaseHTTPRequestHandler):
    def do_GET(self):
        body = json.dumps({'ok': True, 'path': self.path}).encode('utf-8')
        self.send_response(200)
        self.send_header('Content-Type', 'application/json; charset=utf-8')
        self.send_header('Content-Length', str(len(body)))
        self.end_headers()
        self.wfile.write(body)

    def log_message(self, fmt, *args):
        return


server = HTTPServer(('127.0.0.1', 0), Handler)
host, port = server.server_address
Thread(target=server.serve_forever, daemon=True).start()

with urlopen(f'http://{host}:{port}/hello?x=1', timeout=2) as resp:
    payload = resp.read().decode('utf-8')
    print('status:', resp.status)
    print('json:', json.loads(payload))

server.shutdown()


## 知识点 6：超时、重试、幂等、背压：工程化必修

- 超时：所有外部 I/O 都必须设置超时（默认无限等待是事故源）。
- 重试：只对可重试错误重试（超时/瞬时错误）；指数退避+抖动；有上限。
- 幂等：重试会带来重复请求；用幂等键/唯一约束/去重表保证结果不重复。
- 背压/限流：消费者跟不上就必须“减速”，否则队列/内存爆。


## 常见坑

- 把 TCP 当成“消息队列”：一条 send 对应一条 recv（错误）
- 没有超时：网络问题时线程/协程永久挂起
- 无限重试：把故障放大成雪崩
- 不做输入校验：异常数据导致服务端崩溃或安全问题


## 综合小案例：实现“按行分帧”的 TCP 回显服务（本机）

目标：
- 服务端接收以 
 结束的文本行，并回显 `echo: <line>`
- 客户端一次发送多行并读取回显，展示“分隔符分帧”如何规避粘包/半包


In [ ]:
import socket
from threading import Thread


def recv_lines(sock: socket.socket, *, limit=1024 * 64):
    buf = b''
    while True:
        chunk = sock.recv(4096)
        if not chunk:
            break
        buf += chunk
        if len(buf) > limit:
            raise ValueError('line too long')
        while b'\n' in buf:
            line, buf = buf.split(b'\n', 1)
            yield line.decode('utf-8')


def serve(host='127.0.0.1', port=0):
    srv = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    srv.bind((host, port))
    srv.listen(5)
    actual_port = srv.getsockname()[1]
    print('listening on', actual_port)

    def handle(conn: socket.socket):
        with conn:
            for line in recv_lines(conn):
                conn.sendall(('echo: ' + line + '\n').encode('utf-8'))

    def loop():
        while True:
            conn, _addr = srv.accept()
            Thread(target=handle, args=(conn,), daemon=True).start()

    Thread(target=loop, daemon=True).start()
    return srv, actual_port


srv, port = serve()
with socket.create_connection(('127.0.0.1', port), timeout=2) as c:
    c.sendall('a\nb\nc\n'.encode('utf-8'))
    c.shutdown(socket.SHUT_WR)
    data = c.recv(4096).decode('utf-8')
    print(data.strip().splitlines())

srv.close()


## 自测题（不写代码也能回答）

- 为什么 TCP 会出现粘包/半包？
- I/O 密集任务为什么常用线程或 asyncio？CPU 密集为什么常用多进程？
- “幂等”解决的是什么问题？和“重试”有什么关系？


## 练习题（建议写代码）

- 把小案例改成“长度前缀协议”（4 字节大端整数 + payload）。
- 为服务端加超时与最大连接数限制（例如信号量）。
- 实现客户端指数退避重连（最多 3 次）。
